## Explore Dataset -- Data Exploration and Insights Extraction

This notebook aims to explore the dataset -- limited to the training dataset -- to understand the data available and extract insights that might be relevant for the model creation.


### Tasks:
 - [X] Load training dataset.
 - [ ] Understand columns (including target):
     - [X] Values range and distribution.
     - [X] Missing values.
     - [ ] Columns correlation.
 - [X] Identify pre-processing operations needed.
 - [X] Identify potential features and transformations.
 
 
### Dataset Columns

According to the [dataset documentation](https://www.kaggle.com/datasets/sufyant/brazilian-real-bank-dataset), the meaning of each columns is:

 - `id`: id
 - `celular`: phone number
 - `safra_abertura`: branch number
 - `cidade`: city
 - `estado`: state
 - `idade`: age
 - `sexo`: gender
 - `limite_total`: total credit card limit
 - `limite_disp`: current available limit
 - `data`: date
 - `valor`: amount
 - `grupo_estabelecimento`: category of expense 
     - `SERVIÇO`: any service other than restaurants
     - `FAMARCIAS`: drugstores
     - `M.O.T.O`: online purchases
     - `ARTIGOS ELETRO`: electronics
     - `VAREJO`: convenience store
     - `LOJA DE DEPART`: department store
     - `VESTUARIO`: clothing
     - `HOSP E CLINICA`: hospital or clinic
     - `SEM RAMO`: not specified/other expenses
     - `SUPERMERCADOS`: supermarket
     - `MAT CONSTRUCAO`: construction material
     - `POSTO DE GAS`: gas station
     - `RESTAURANTE`: restaurant
     - `CIA AEREAS`: airlines
     - `MOVEIS E DECOR`: furniture & decoration
     - `JOALHERIA`: jewelry
     - `TRANS FINANC`: financial transfers
     - `AGENCIA DE TUR`: tourism agency
     - `HOTEIS`: hotels
 - `cidade_estabelecimento`: city where the purchase was made
 - `pais_estabelecimento`: country where the purchase was made
 
 The column `celular`, however, was not found. Maybe it was removed due to privacy concerns. 

## Libraries and Configurations

In [1]:
from datetime import datetime
from typing import List, Dict

import numpy as np
import pandas as pd
from IPython.core.display import HTML

from application.code.core.configurations import configs
from application.code.adapters.storage import read_dataset
from application.code.core.feature_engineering import (clean_column_names,
                                                       change_column_types,
                                                       format_string_columns, 
                                                       engineer_features)

## Constants

In [2]:
NUMERIC_COLUMNS = ['safra_abertura',  'idade', 
                   'limite_total', 'limite_disp',  'valor',                   
                  ]

CATEGORICAL_COLUMNS = ['id', 'cidade', 'estado', 'sexo', 'data',
                       'grupo_estabelecimento', 'cidade_estabelecimento',
                       'pais_estabelecimento']

## Local Functions

In [3]:
def count_missing_values(df: pd.DataFrame) -> List[Dict[str, int]]:

    missing_values = []

    for column in df.columns:

        columns_missing_values = len(df[lambda f: f[column].isna() |
                                       f[column].apply(lambda col: (len(str(col).strip()) == 0) or (str(col) == '-'))])

        missing_values.append({'column': column, 
                               'missing_values': columns_missing_values})
        
       
    return missing_values

## Load Dataset

The `training` dataset is loaded to allow the analysis.

In [4]:
df = read_dataset(base_path=configs.datasets.base_path, stage='raw', file_name='train')

print(f'Records: {len(df)}')

print('\nSample:')
df.head(3).T

Records: 3944

Sample:


,0,1,2
id,"4,53E+11","4,53E+11","4,53E+11"
safra_abertura,201405,201405,201405
cidade,CAMPO LIMPO PAULISTA,CAMPO LIMPO PAULISTA,CAMPO LIMPO PAULISTA
estado,SP,SP,SP
idade,37,37,37
sexo,F,F,F
limite_total,4700,4700,4700
limite_disp,5605,5343,2829
data,4.12.2019,9.11.2019,6.05.2019
valor,31,"150,01",50


### Basic Dataset Preprocessing

During the analysis some basic transformations and fixes were identified. To avoid dealing with the issues along the notebook, the most basic ones will be performed in this section.

In [5]:
df = (
    df
    .pipe(clean_column_names)
    .pipe(change_column_types)
    .pipe(format_string_columns, columns=CATEGORICAL_COLUMNS)
)

## Exploratory Data Analysis

### Columns Summary

In [6]:
display(HTML('<h4>Numeric Columns</h4>'))
display(df[NUMERIC_COLUMNS].describe(include='all'))

display(HTML('<h4>Categorical Columns</h4>'))
display(df[CATEGORICAL_COLUMNS].describe(include='all'))

,safra_abertura,idade,limite_total,limite_disp,valor
count,3944.000000,3944.000000,3944.000000,3944.000000,3933.000000
mean,201681.928499,33.818966,8240.568458,6326.599899,103.132319
std,225.684776,6.857314,9925.155290,9820.570467,393.700590
min,201008.000000,20.000000,300.000000,0.000000,0.270000
25%,201602.000000,28.000000,3650.000000,1082.000000,13.280000
50%,201710.000000,33.000000,4700.000000,2676.500000,30.000000
75%,201901.000000,36.000000,7500.000000,6012.250000,81.350000
max,201910.000000,53.000000,40000.000000,42289.000000,10602.000000


,id,cidade,estado,sexo,data,grupo_estabelecimento,cidade_estabelecimento,pais_estabelecimento
count,3944,3944,3944,3944,3944,3944,3944,3944
unique,28,10,2,2,301,21,115,8
top,"5,02e+11",sao paulo,sp,f,10.08.2019,servio,sao paulo,br
freq,587,2960,3734,2097,32,1455,2624,3914


#### Categorical Columns Detailing

This section explores the categorical values from columns. When there are more then 10 distinct values for a column, an scroll is enabled and the background color is set to blue.

In [7]:
for column in CATEGORICAL_COLUMNS:
    
    distinct_values = sorted(df[column].dropna().unique().tolist())
    serialized_values = ''.join([f'<li>{value}</li>' for value in distinct_values])
    
    display(HTML(f'<strong>{column} - {len(distinct_values)}</strong>'))
    
    if len(distinct_values) > 10:
        style = style=("style='height:250px;width:500px;"
                       "overflow:scroll;background-color:#d4e6f1;'")
    else:
        style = ''
    
    display(HTML(f'''
    <div {style}>
        <ul>{serialized_values}</ul>
    </div>
    '''))

Detailing the cases where `pais_estabelecimento` is not `br` and `cidade` is in `cidade_estabelecimento`.

In [8]:
brazilian_cities = set(df['cidade'].tolist())

(
    df
    .loc[lambda f: (f['cidade_estabelecimento'].isin(brazilian_cities)) & 
                   (f['pais_estabelecimento'] != 'br')]
)

,id,safra_abertura,cidade,estado,idade,sexo,limite_total,limite_disp,data,valor,grupo_estabelecimento,cidade_estabelecimento,pais_estabelecimento,period
3377,64831300521,201008,itaquaquecetuba,sp,51,m,1420,580,23.01.2020,NaN,varejo,sao paulo,sp,2020-01-23


About categorical values, it is possible to identify that:
 - `cidade` and `estado`, which are users' location, have a limited and consistent number of values.
 - `cidade_estabelecimento` and `pais_estabelecimento` are more diversified and present some inconsistent values, such as `11001bogota` and `415-503-9235`. Some of those values might be recorded for the wrong column, such as `5,51e+11`. Also, both of them present missing values. 
 - `grupo_estabelecimento`, which is the label to be learned and predicted, is consistent.

Some approaches that could be performed to improve the dataset quality:
 - Infer inconsistent values based users' transactions performed in a limited period of time.
 - Use a map of cities, states and countries to validate and infer values.

 
Due to the time limit to conduct this project and the potentially negligible number of records, no treatment will be performed initially.

### Missing Values

This section checks, for each column, the number of missing values (e.g., `na`, `nan`, `''`, and `-`).

In [9]:
missing_values = count_missing_values(df)

(
    pd.DataFrame(missing_values)
    .sort_values(by=['missing_values', 'column'], ascending=[False, True])
)

,column,missing_values
11,cidade_estabelecimento,11
9,valor,11
12,pais_estabelecimento,10
2,cidade,0
8,data,0
3,estado,0
10,grupo_estabelecimento,0
0,id,0
4,idade,0
7,limite_disp,0


As there are three columns with missing values, the incomplete records will be checked.

In [10]:
df_slice = df.loc[lambda f: f['pais_estabelecimento'].isna() | 
                            f['cidade_estabelecimento'].isna() | 
                            f['valor'].isna()]

print(f'Records with missing values: {len(df_slice)} ({len(df_slice) / len(df) * 100:.2f}%)')
df_slice

Records with missing values: 11 (0.28%)


,id,safra_abertura,cidade,estado,idade,sexo,limite_total,limite_disp,data,valor,grupo_estabelecimento,cidade_estabelecimento,pais_estabelecimento,period
572,94873707154,201710,sao paulo,sp,32,f,2200,628,12.05.2019,NaN,trans financ,,,2019-05-12
629,94873707154,201710,sao paulo,sp,32,f,2200,9,10.09.2019,NaN,trans financ,,,2019-09-10
868,"5,02e+11",201901,sao paulo,sp,34,f,5750,1012,27.12.2019,NaN,trans financ,,,2019-12-27
1123,"5,02e+11",201901,sao paulo,sp,34,f,3400,110,13.08.2019,NaN,trans financ,,,2019-08-13
1166,"5,02e+11",201901,sao paulo,sp,34,f,4000,934,19.10.2019,NaN,trans financ,,,2019-10-19
1170,"5,02e+11",201901,sao paulo,sp,34,f,4000,333,22.10.2019,NaN,trans financ,,,2019-10-22
2738,"5,02e+11",201901,sao paulo,sp,34,f,5200,36,27.08.2019,NaN,trans financ,,,2019-08-27
2759,"5,02e+11",201901,sao paulo,sp,34,f,5200,2004,10.11.2019,NaN,trans financ,,,2019-11-10
2767,"5,02e+11",201901,sao paulo,sp,34,f,6500,955,31.08.2019,NaN,trans financ,,,2019-08-31
2983,"8,14e+11",201705,sao paulo,sp,28,m,4700,2102,12.04.2019,NaN,trans financ,,,2019-04-12


About missing values, it is possible to observe:
 - The number of missing values 12 is not a problem in terms of training a model, as it only influences 0.3% of the data.
 - All missing values from `pais_estabelecimento` are also missing in `cidade_estabelecimento`. Maybe `cidade` and `estado` can be used to infer the missing values.
 - Missing values for column `valor` are also highly related to missing values on `establishment` columns. It might be a quality from the organization responsible for sending the data.
 
 The presence of missing values on training might be an indicative that production data might also have missing values. It would be interesting to know what to do in terms of product. Some options are:
 - Do not use features with missing values.
 - Do not classify data with missing values.
 - Perform generic data imputation for missing values.
 - Use data from a recent user's history to infer the location (using some time distance threshold).
 - Use an algorithm that deals with missing values.

## Feature Engineering

Some functions were created to derive new features from the available columns. The features created were:
 
 - `estado_estabelecimento`: Infers the establishment state from the establishment city using users' cities and states.
 - `is_a_different_city`: Indicate if user city is different from establishment city (might indicate a travel).
 - `is_a_different_state`: Indicate if user state is different from establishment state (might indicate a travel).
 - `is_a_different_country`: Indicate if user country is different from establishment state (might indicate a travel).
 - `weekday`: Extract the weekday from the date (might provide information about the type of transaction).
 - `monthday`: Extract the month day from the date (might provide information about the type of transaction).
 - `month`: Extract the month from the date (might provide information about the type of transaction).
 - `total_relative_value`: Indicate the percentage of the value from the total value.
 - `available_relative_value`: Indicate the percentage of the value from the available value.

In [11]:
(
    df
    .sample(5)
    .pipe(engineer_features)
    .T
)

,3744,1098,724,2495,1203
id,"5,38e+11","5,02e+11","5,43e+11","8,14e+11","5,02e+11"
safra_abertura,201905,201901,201807,201705,201901
cidade,santo andre,sao paulo,sao paulo,sao paulo,sao paulo
estado,sp,sp,sp,sp,sp
idade,28,34,32,28,34
sexo,f,f,f,m,f
limite_total,4200,3750,4200,4700,5200
limite_disp,2196,563,4996,3286,199
data,9.10.2019,27.09.2019,23.01.2020,2.07.2019,27.08.2019
valor,39.9,24.5,22.6,4.0,37.8


There are some other potential features that could be used, but would require some additional efforts:
 - Some time window based features:
     - Average value of last transactions.
     - Number of transactions on the last `n` days.
     - Average number of different cities on the last transactions.
 - Demographic information about user and establishment cities.
 - User cluster, considering expenses, age, gender, etc.     

## Conclusions

The main findings were:
 - There are some missing and missplacing values that degrades the dataset quality. The number of issues is not excessive, but I might require some additional treatments when performing production/test predictions.
 - Some additional features were derived based on location, date, limits and values.
 - As the new features were created and there are some quality issues, it might be relevant to perform a feature importance analysis.
 
## Future Activities

Some additional possibilities are:
 - Compute features correlation.
 - Analyze the history of some users to identify new possible features and patterns.
 - Analyze the transaction types along time.